In [5]:
import pandas as pd
import pm4py
from tqdm import tqdm
import os
import numpy as np

In [2]:
#  ALREADY RAN
for file in os.listdir('../data/test_test_data/'):
    in_path = '../data/train_test_data/' + file
    out_path = '../data/' + file.replace('.xes', '.csv')
    log = pm4py.read_xes(in_path)
    df = pm4py.convert_to_dataframe(log)
    df.to_csv(out_path, index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../data/test_test_data/'

In [6]:
# declined_train = pd.read_csv('../data/declined_train.csv')
declined_test = pd.read_csv('../data/declined_test.csv', index_col=0).reset_index(drop=True)
print(len(declined_test['case:concept:name'].unique()))

declined_train = pd.read_csv('../data/declined_train.csv', index_col=0).reset_index(drop=True)
print(len(declined_train['case:concept:name'].unique()))

751
3001


In [7]:
approved_train = pd.read_csv('../data/approved_train.csv', index_col=0).reset_index(drop=True)
approved_test = pd.read_csv('../data/approved_test.csv', index_col=0).reset_index(drop=True)



cancelled_train = pd.read_csv('../data/cancelled_train.csv', index_col=0).reset_index(drop=True)
cancelled_test = pd.read_csv('../data/cancelled_test.csv', index_col=0).reset_index(drop=True)

In [8]:
from random import sample
train = pd.concat([declined_train, approved_train[approved_train['case:concept:name'].isin(sample(list(approved_train['case:concept:name'].unique()), 3000))], cancelled_train[cancelled_train['case:concept:name'].isin(sample(list(cancelled_train['case:concept:name'].unique()), 3000))]])
train['event_w_lifecycle'] = train['concept:name'] + ' ' + train['lifecycle:transition']
train 

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,event_w_lifecycle
0,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,A_Create Application complete
1,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,A_Submitted complete
2,Created,User_1,W_Handle leads,Workflow,Workitem_831373279,schedule,2016-01-01 10:16:11.740000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Handle leads schedule
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1299098074,withdraw,2016-01-01 10:17:31.573000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Handle leads withdraw
4,Created,User_1,W_Complete application,Workflow,Workitem_1703931302,schedule,2016-01-01 10:17:31.584000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Complete application schedule
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219079,statechange,User_3,A_Complete,Application,ApplState_546159617,complete,2016-11-08 13:08:30.990000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Motorcycle,New credit,Application_28299345,5000.0,A_Complete complete
219080,Released,User_3,W_Call after offers,Workflow,Workitem_1711586493,suspend,2016-11-08 13:10:12.110000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Motorcycle,New credit,Application_28299345,5000.0,W_Call after offers suspend
219081,statechange,User_49,A_Cancelled,Application,ApplState_2067070380,complete,2016-11-09 11:47:05.252000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Motorcycle,New credit,Application_28299345,5000.0,A_Cancelled complete
219082,statechange,User_49,O_Cancelled,Offer,OfferState_1317004789,complete,2016-11-09 11:47:05.271000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_947561374,Motorcycle,New credit,Application_28299345,5000.0,O_Cancelled complete


In [9]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping-dummies to create the the categorical encoding
# https://stackoverflow.com/questions/49161120/pandas-python-set-value-of-one-column-based-on-value-in-another-column to assign values of the result DF
# https://stackoverflow.com/questions/71426679/cumulative-sum-of-time-from-timestamps-in-pandas for cumulative time for boris

In [10]:
org_resource_cols = list(train['org:resource'].unique())
event_cols = list(train['event_w_lifecycle'].unique())
rest_cols = ['FirstWithdrawalAmount',
       'NumberOfTerms', 'Accepted', 'MonthlyCost', 'Selected', 'CreditScore',
       'OfferedAmount', 'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount']
target_cols = ['first_timestamp', 'trace_duration', 'case_outcome']

res_cols = org_resource_cols + event_cols + rest_cols + target_cols


In [11]:
tester = train[train['case:concept:name'] == 'Application_1691306052']
tester

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,event_w_lifecycle
0,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,A_Create Application complete
1,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,A_Submitted complete
2,Created,User_1,W_Handle leads,Workflow,Workitem_831373279,schedule,2016-01-01 10:16:11.740000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Handle leads schedule
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1299098074,withdraw,2016-01-01 10:17:31.573000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Handle leads withdraw
4,Created,User_1,W_Complete application,Workflow,Workitem_1703931302,schedule,2016-01-01 10:17:31.584000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Complete application schedule
5,statechange,User_1,A_Concept,Application,ApplState_421204851,complete,2016-01-01 10:17:31.594000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,A_Concept complete
6,Obtained,User_17,W_Complete application,Workflow,Workitem_1746553641,start,2016-01-02 10:50:42.796000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Complete application start
7,Released,User_17,W_Complete application,Workflow,Workitem_1713535355,suspend,2016-01-02 10:51:27.582000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Complete application suspend
8,Obtained,User_38,W_Complete application,Workflow,Workitem_1495694398,resume,2016-01-02 10:53:34.108000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Complete application resume
9,statechange,User_38,A_Accepted,Application,ApplState_525970231,complete,2016-01-02 10:59:44.735000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,A_Accepted complete


In [16]:
def aggregate_df(df, res_cols):
    """ 
    Aggregate the df of current events in the case

    Output: 
        result -> could be a pandas series
    """
    res_dict = dict.fromkeys(res_cols)
    
    for row in df.to_dict('records'):

        # user variable assign
        user = row['org:resource']
        # first mention, turn it numeric
        if not res_dict[user]: res_dict[user] = 1
        else: res_dict[user] += 1
    
        # event variable assign
        event = row['concept:name'] + ' ' + row['lifecycle:transition']
        if not res_dict[event]: res_dict[event] = 1
        else: res_dict[event] += 1

        # case level cols
        rest_cols = ['FirstWithdrawalAmount',
       'NumberOfTerms', 'Accepted', 'MonthlyCost', 'Selected', 'CreditScore',
       'OfferedAmount', 'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount'] 

        for col in rest_cols:
            res_dict[col] = row[col]
        
        # time stuff
        #keeping the first timestamp of the case for calculation purposes
        if not res_dict['first_timestamp']: res_dict['first_timestamp'] = row['time:timestamp']

        # trace duration in seconds
        res_dict['trace_duration'] = (np.datetime64(row['time:timestamp']) - np.datetime64(res_dict['first_timestamp'])).item().total_seconds()

    return pd.DataFrame.from_dict([res_dict])



In [17]:
def add_to_aggregate(result, df_row):
    """ 
    When a new event happens, add the event info to the current aggregated result.

    Input: 
         result: the current aggregated result
         df_row: pandas df row representing the new event
    Output:
        result: the new aggregated result
    """
    last_row = result.to_dict('records')[-1]
    new_event = df_row.to_dict()
    new_row = last_row

    # user variable assign
    user = new_event['org:resource']

    # first mention, turn it numeric
    if not new_row[user]: new_row[user] = 1
    else: new_row[user] += 1
    # event variable assign
    event = new_event['concept:name'] + ' ' + new_event['lifecycle:transition']
    if not new_row[event]: new_row[event] = 1
    else: new_row[event] += 1

    # case level cols
    rest_cols = ['FirstWithdrawalAmount',
    'NumberOfTerms', 'Accepted', 'MonthlyCost', 'Selected', 'CreditScore',
    'OfferedAmount', 'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount'] 

    for col in rest_cols:
        new_row[col] = new_event[col]
    
    # time stuff
    # trace duration in seconds
    new_row['trace_duration'] = (np.datetime64(new_event['time:timestamp']) - np.datetime64(new_row['first_timestamp'])).item().total_seconds()
    return new_row

In [18]:
def create_prefix_part2(cases_df, res_cols, end_event, start_event='A_Accepted'):
    
    app_id_list = list(cases_df['case:concept:name'].unique())


    # create a return df
    res = pd.DataFrame(columns=res_cols)

    # extracting prefix for each application
    for app_id in app_id_list:
        
        events_app = cases_df.loc[cases_df['case:concept:name'] == app_id]
        events_app.reset_index(drop=True, inplace=True)

        cur_id = starting_row_id = events_app.loc[events_app['concept:name'] == start_event].index[0] + 1
        pre_events = events_app.iloc[:starting_row_id]

        # aggregate events_app from row 0 to starting_row_id
        agg = aggregate_df(pre_events, res_cols=res_cols)

        res = pd.concat([res, agg])

        ending_row_id = events_app.loc[events_app['concept:name'] == end_event].index[0] 
        while cur_id < ending_row_id:
            new_row = events_app.iloc[cur_id]

            # add new event row info to the aggregated result
            new_dict = add_to_aggregate(res, new_row)
            df = pd.DataFrame.from_dict([new_dict])
            res = pd.concat([res, df])
            # Update the return_df -> add new row
            # target y: end_event
            cur_id += 1
        res['case_outcome'] = end_event

        return res

In [19]:
create_prefix_part2(tester, res_cols, 'A_Denied')

/var/folders/9l/kwd1671542z9tyvsgbp6mmtr0000gn/T/ipykernel_88563/3781704134.py:36: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  res_dict['trace_duration'] = (np.datetime64(row['time:timestamp']) - np.datetime64(res_dict['first_timestamp'])).item().total_seconds()
/var/folders/9l/kwd1671542z9tyvsgbp6mmtr0000gn/T/ipykernel_88563/2406954911.py:36: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  new_row['trace_duration'] = (np.datetime64(new_event['time:timestamp']) - np.datetime64(new_row['first_timestamp'])).item().total_seconds()
/var/folders/9l/kwd1671542z9tyvsgbp6mmtr0000gn/T/ipykernel_88563/1093703705.py:30: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to

,User_1,User_17,User_38,User_11,User_119,User_30,User_68,User_3,User_113,User_95,...,MonthlyCost,Selected,CreditScore,OfferedAmount,case:LoanGoal,case:ApplicationType,case:RequestedAmount,first_timestamp,trace_duration,case_outcome
0,6,2,2,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,89013.235,A_Denied
0,6,2,3,None,None,None,None,None,None,None,...,200.0,0.0,0.0,6000.0,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,89187.494,A_Denied
0,6,2,4,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,89188.942,A_Denied
0,6,2,5,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,89213.800,A_Denied
0,6,2,6,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,89213.824,A_Denied
0,6,2,7,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,89213.837,A_Denied
0,6,2,8,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,89213.843,A_Denied
0,6,2,9,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,89213.846,A_Denied
0,6,2,10,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,89584.153,A_Denied
0,6,2,10,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,Home improvement,New credit,10000.0,2016-01-01 10:16:11.500000+00:00,427663.464,A_Denied
